In [1]:
# %matplotlib ipympl
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *

In [4]:
fig = plt.figure(figsize=(8, 6))
# fig.show()

In [5]:
data = []

config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [True, True, True, True],
    "gpio_delay_time" : 0.010795,
    "augment_gpio": True,
    "plot_pins_method": "line",
    "plot_xmax": 1.2,
    "fig": fig,
}

fig.clf()
for ax in fig.get_axes():
    ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(1)

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0.0
  ax.set_xlim((valmin, valmax))
D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0.0
  ax.set_xlim((valmin, valmax))


In [6]:
print(data.gpio)

((0.012177933333333335, [False, False, False, False]), (0.012177933333333335, [True, True, True, True]), (0.0125438, [True, True, True, True]), (0.0125438, [True, True, False, True]), (0.0125486, [True, True, False, True]), (0.0125486, [True, True, False, False]), (0.03739873333333334, [True, True, False, False]), (0.03739873333333334, [True, True, True, False]), (0.038400866666666665, [True, True, True, False]), (0.038400866666666665, [True, True, False, False]), (0.0484846, [True, True, False, False]), (0.0484846, [True, True, False, True]), (0.0495358, [True, True, False, True]), (0.0495358, [True, True, False, False]), (0.0698126, [True, True, False, False]), (0.0698126, [True, True, True, False]), (0.07179713333333333, [True, True, True, False]), (0.07179713333333333, [True, True, False, False]), (0.08188140000000001, [True, True, False, False]), (0.08188140000000001, [True, True, False, True]), (0.08396513333333333, [True, True, False, True]), (0.08396513333333333, [True, True, F